# 展平层

在小明的冰激凌店的预测模型里，每个数据都是一个向量（一维数组)，比如：[28.1, 58.0]。而图像数据至少是二维（高、宽）数据。在MINST数据集里，我们更是把图像数据扩展到三维，增加了一个关于颜色的维度。

那么问题来了：线性层只能处理一维的特征值，怎样让它处理我们多维的图像数据呢？

答案其实很简单，就是使用**展平层**（Flatten Layer）把无论多少维度的数据，都重新排列成一行，变成一个向量（一维数组）。

```{figure} images/flatten.png
:align: center
:width: 480px
**图例：展平层示意图**
```

这样，我们就可以使用线性层来处理图像数据了。

In [1]:
from abc import abstractmethod, ABC
import numpy as np

np.random.seed(99)

## 基础架构

### 张量

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = np.zeros_like(self.data)
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def shape(self):
        return self.data.shape

    def size(self):
        return np.prod(self.data.shape[1:])

    def __str__(self):
        return f'Tensor({self.data})'

### 基础数据集

In [3]:
class Dataset(ABC):

    def __init__(self, batch_size=1):
        self.batch_size = batch_size
        self.load()
        self.train()

    @abstractmethod
    def load(self):
        pass

    def train(self):
        self.features = self.train_features
        self.labels = self.train_labels

    def eval(self):
        self.features = self.test_features
        self.labels = self.test_labels

    def shape(self):
        return Tensor(self.features).size(), Tensor(self.labels).size()

    def items(self):
        return Tensor(self.features), Tensor(self.labels)

    def __len__(self):
        return len(self.features) // self.batch_size

    def __getitem__(self, index):
        start = index * self.batch_size
        end = start + self.batch_size

        feature = Tensor(self.features[start: end])
        label = Tensor(self.labels[start: end])
        return feature, label

### 基础层

In [4]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### 基础损失函数

In [5]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### 基础优化器

In [6]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def reset(self):
        for p in self.parameters:
            p.grad = np.zeros_like(p.data)

    @abstractmethod
    def step(self):
        pass

### 基础模型

In [7]:
class Model(ABC):

    def __init__(self, layer, loss_fn, optimizer):
        self.layer = layer
        self.loss_fn = loss_fn
        self.optimizer = optimizer

    @abstractmethod
    def train(self, dataset, epochs):
        pass

    @abstractmethod
    def test(self, dataset):
        pass

## 数据

### MINST数据集

我们在MNIST数据集里添加一个评估函数estimate()，用来评估模型预测的准确性。

MNIST数据集的标签值是从0到9十个数字。我们已经把标签值通过独热编码转换成一个长度为10的向量，比如2的独热编码是X=[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]。对应的，模型推理的预测值也将是一个长度为10的向量。那么我们观察预测值向量的最大值在哪个位置，就可以知道模型预测出的数字是几。比如，如果预测值是P=[0.01, 0.02, 0.81, 0.05, 0.01, 0.02, 0.04, 0.01, 0.02, 0.01]，那么最大值是P[2]，因此模型预测的结果就是2。

我们统计出有多少测试数据的预测值和标签值一样，就可以得出模型测试的准确率。

In [8]:
class MINSTDataset(Dataset):

    def __init__(self, filename, batch_size=1):
        self.filename = filename
        super().__init__(batch_size)

    def load(self):
        with (np.load(self.filename, allow_pickle=True) as f):
            self.train_features, self.train_labels = self.normalize(f['x_train'], f['y_train'])
            self.test_features, self.test_labels = self.normalize(f['x_test'], f['y_test'])

    @staticmethod
    def normalize(x, y):
        inputs = x / 255
        inputs = np.expand_dims(inputs, axis=1)
        targets = np.zeros((len(y), 10))
        targets[range(len(y)), y] = 1
        return inputs, targets

    def estimate(self, predictions):
        count = (predictions.data.argmax(axis=1) == self.labels.argmax(axis=1)).sum()
        total = len(self.labels)
        return count / total

## 模型

### 线性层

In [9]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.random.rand(out_size, in_size) / in_size)
        self.bias = Tensor(np.random.rand(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad.T @ x.data / len(x.data)
            self.bias.grad += np.sum(p.grad, axis=0) / len(x.data)
            x.grad += p.grad @ self.weight.data / len(x.data)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 顺序层

In [10]:
class Sequential(Layer):

    def __init__(self, layers):
        self.layers = layers

    def forward(self, x: Tensor):
        for l in self.layers:
            x = l(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

    def __str__(self):
        return '\n'.join(str(l) for l in self.layers if str(l))

### 展平层

前向传播：使用NumPy的reshape()函数，我们可以在不改变数据的实际位置的情况下，直接改变数组的维度形状。

梯度计算：在梯度计算函数gradient_fn()里，我们可以用同样的方法，把后一层传回來的梯度重现转换成原来的维度形状。这样就可以继续向前一层传递了。

In [11]:
class Flatten(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.array(x.data.reshape(x.data.shape[0], -1)))

        def gradient_fn():
            x.grad += p.grad.reshape(x.data.shape)

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'Flatten[]'

### ReLU激活函数

In [12]:
class ReLU(Layer):

    def forward(self, x: Tensor):
        p = Tensor(np.maximum(0, x.data))

        def gradient_fn():
            x.grad += p.grad * (p.data > 0) 

        p.gradient_fn = gradient_fn
        p.parents = {x}
        return p

    def __str__(self):
        return f'ReLU[]'

### 损失函数（平均平方差）

In [13]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### 优化器（随机梯度下降）

In [14]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

### 神经元网络模型

In [15]:
class NNModel(Model):

    def train(self, dataset, epochs):
        dataset.train()

        for epoch in range(epochs):
            for i in range(len(dataset)):
                features, labels = dataset[i]

                predictions = self.layer(features)
                loss = self.loss_fn(predictions, labels)

                self.optimizer.reset()
                loss.backward()
                self.optimizer.step()

    def test(self, dataset):
        dataset.eval()

        features, labels = dataset.items()
        predictions = self.layer(features)
        loss = self.loss_fn(predictions, labels)
        return predictions, loss

## 设置

### 学习率

In [16]:
LEARNING_RATE = 0.01

### 批大小

In [17]:
BATCH_SIZE = 2

### 轮次

In [18]:
EPOCHS = 10

## 训练

### 迭代

我们的第一个图像识别网络模型首先使用展平层，用来把图像数据转换成向量数据。然后连接两个线性层：
* 隐藏层：我们使用64个人工神经元。
* 输出层：必须是10个人工神经元，对应标签值的独热编码。

In [19]:
dataset = MINSTDataset('tinymnist.npz', BATCH_SIZE)
layer = Sequential([Flatten(),
                    Linear(dataset.shape()[0], 64),
                    ReLU(),
                    Linear(64, dataset.shape()[1])])
loss_fn = MSELoss()
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

model = NNModel(layer, loss_fn, optimizer)
model.train(dataset, EPOCHS)
print(layer)

Flatten[]
Linear[weight(64, 784); bias(64,)]
ReLU[]
Linear[weight(10, 64); bias(10,)]


## 验证

### 测试

我们使用的微型数据集TinyMNIST只有2000个数据样本。经过50轮的训练，看看我们的模型的预测准确率能达到多少。

In [20]:
predictions, loss = model.test(dataset)
accuracy = dataset.estimate(predictions)
print(f'accuracy: {accuracy:.2%}')

accuracy: 92.20%


这已经是一个很不错的准确率了。